# Tax-free childcare

[Official government guidance](https://www.gov.uk/tax-free-childcare?step-by-step-nav=d78aeaf6-1747-4d72-9619-f16efb4dd89d) | [Legislation](https://www.legislation.gov.uk/ukdsi/2015/9780111127063)

## Program description

### Overview

Tax-free childcare helps working families with childcare costs. For every **£8** paid into a childcare account, the government adds **£2**.

The program provides financial support up to **£2,000** per year (**£500** per quarter) for each eligible child. For disabled children, this support increases to **£4,000** per year (**£1,000** per quarter).

### Eligibility requirements

#### Age criteria
Standard childcare support extends until **September 1st** following the child's **11th** birthday. For **disabled children** who receive disability benefits or are certified as **blind/severely sight-impaired**, support continues until **September 1st** following their **16th** birthday.

#### Income requirements
Income thresholds apply at both ends of the spectrum. The maximum adjusted net income allowed is **£100,000** per year for each partner. Minimum earnings requirements (per **3 months**) vary by age:

| Age group | Minimum earnings |
|-----------|-----------------|
| **21+** | **£2,379** |
| **18-20** | **£1,788** |
| **Under 18/Apprentices** | **£1,331** |

#### Work status
Eligible individuals must be either **employed**, **self-employed**, or on qualifying leave (**sick**, **annual**, or **shared parental leave**). Applications can be submitted if starting work within **31 days**.

Non-working partners may still qualify if they receive any of these benefits: **incapacity benefit**, **severe disablement allowance**, **carer's allowance**, or **contribution-based employment and support allowance**.

### Additional rules

#### Income assessment
The program offers flexible income assessment:
* Combines **multiple income sources**
* Allows **annual income averaging**
* Gives **self-employed** individuals choice between total or self-employment income

Certain types of income are excluded from calculations: **dividends**, **interest**, **property income**, and **pension payments**.

#### Program interactions
Tax-free childcare is compatible with the **15/30 hours free childcare** program but cannot be combined with:
* **Working tax credit**
* **Child tax credit**
* **Universal credit**
* **Childcare vouchers** or directly contracted childcare

When approved for tax-free childcare:
* Existing benefits terminate automatically
* Childcare vouchers must be cancelled within **90 days**
* **New self-employed** businesses (under **12 months**) benefit from more flexible earnings requirements

## Implementation

In the following steps, we explain how PolicyEngine models the tax-free childcare program through parameter definitions and calculations.

### 1. Age determination
   First, we determine which children in the household are age-eligible for the program, as tax-free childcare is only available until specific age thresholds.
   * [`standard.yaml`](https://github.com/PolicyEngine/policyengine-uk/blob/16cc0f012bdccd0a20d445aa44b8e47906b04059/policyengine_uk/parameters/gov/hmrc/tax_free_childcare/age/standard.yaml) and [`disability.yaml`](https://github.com/PolicyEngine/policyengine-uk/blob/16cc0f012bdccd0a20d445aa44b8e47906b04059/policyengine_uk/parameters/gov/hmrc/tax_free_childcare/age/disability.yaml) define age thresholds
   * [`tax_free_childcare_child_age_eligible.py`](https://github.com/PolicyEngine/policyengine-uk/blob/16cc0f012bdccd0a20d445aa44b8e47906b04059/policyengine_uk/variables/gov/hmrc/tax_free_childcare/conditions/tax_free_childcare_child_age_eligible.py) calculates whether each child meets the standard (11 years) or disability-based (16 years) age requirements

### 2. Income assessment
   After identifying eligible children, we evaluate if the parents meet the income criteria, which includes both minimum and maximum thresholds.
   * [`countable_sources.yaml`](https://github.com/PolicyEngine/policyengine-uk/blob/16cc0f012bdccd0a20d445aa44b8e47906b04059/policyengine_uk/parameters/gov/hmrc/tax_free_childcare/income/countable_sources.yaml) specifies which income types count (employment and self-employment)
   * [`income_limit.yaml`](https://github.com/PolicyEngine/policyengine-uk/blob/16cc0f012bdccd0a20d445aa44b8e47906b04059/policyengine_uk/parameters/gov/hmrc/tax_free_childcare/income/income_limit.yaml) sets the £100,000 maximum threshold
   * [`minimum_weekly_hours.yaml`](https://github.com/PolicyEngine/policyengine-uk/blob/16cc0f012bdccd0a20d445aa44b8e47906b04059/policyengine_uk/parameters/gov/hmrc/tax_free_childcare/minimum_weekly_hours.yaml) defines the 16-hour work requirement
   * [`tax_free_childcare_meets_income_requirements.py`](https://github.com/PolicyEngine/policyengine-uk/blob/16cc0f012bdccd0a20d445aa44b8e47906b04059/policyengine_uk/variables/gov/hmrc/tax_free_childcare/conditions/tax_free_childcare_meets_income_requirements.py) uses these to calculate:
     - Total eligible income from countable sources
     - Quarterly income requirements
     - Compliance with maximum thresholds

### 3. Program compatibility
   We then check if the household receives any benefits that would make them ineligible for tax-free childcare, as certain benefits cannot be claimed alongside it.
   * [`disqualifying_benefits.yaml`](https://github.com/PolicyEngine/policyengine-uk/blob/16cc0f012bdccd0a20d445aa44b8e47906b04059/policyengine_uk/parameters/gov/hmrc/tax_free_childcare/disqualifying_benefits.yaml) lists incompatible benefits
   * [`tax_free_childcare_program_eligible.py`](https://github.com/PolicyEngine/policyengine-uk/blob/16cc0f012bdccd0a20d445aa44b8e47906b04059/policyengine_uk/variables/gov/hmrc/tax_free_childcare/conditions/tax_free_childcare_program_eligible.py) checks if the household receives any disqualifying benefits like Working Tax Credit

### 4. Work status
   The program requires parents to be working, so we verify employment status with special provisions for couples where one partner has a disability.
   * [`tax_free_childcare_work_condition.py`](https://github.com/PolicyEngine/policyengine-uk/blob/16cc0f012bdccd0a20d445aa44b8e47906b04059/policyengine_uk/variables/gov/hmrc/tax_free_childcare/conditions/tax_free_childcare_work_condition.py) evaluates:
     - Single parents: must be working
     - Couples: either both working or one working with partner receiving disability benefits
     - Links with disability benefit variables to confirm eligibility

### 5. Final eligibility
   Now we combine all previous eligibility checks to determine if the household qualifies for the program overall.
   * [`tax_free_childcare_eligibility.py`](https://github.com/PolicyEngine/policyengine-uk/blob/16cc0f012bdccd0a20d445aa44b8e47906b04059/policyengine_uk/variables/gov/hmrc/tax_free_childcare/tax_free_childcare_eligibility.py) combines:
     - Age eligibility results
     - Income requirement checks
     - Program compatibility verification
     - Work condition compliance
   * Determines overall program eligibility

### 6. Contribution amount
   Finally, for eligible households, we calculate the amount of support they can receive based on the number and status of qualifying children.
   * [`standard_child.yaml`](https://github.com/PolicyEngine/policyengine-uk/blob/16cc0f012bdccd0a20d445aa44b8e47906b04059/policyengine_uk/parameters/gov/hmrc/tax_free_childcare/contribution/standard_child.yaml) and [`disabled_child.yaml`](https://github.com/PolicyEngine/policyengine-uk/blob/16cc0f012bdccd0a20d445aa44b8e47906b04059/policyengine_uk/parameters/gov/hmrc/tax_free_childcare/contribution/disabled_child.yaml) define maximum benefits
   * [`tax_free_childcare_benefits_amount.py`](https://github.com/PolicyEngine/policyengine-uk/blob/16cc0f012bdccd0a20d445aa44b8e47906b04059/policyengine_uk/variables/gov/hmrc/tax_free_childcare/tax_free_childcare_benefits_amount.py):
     - Identifies eligible children
     - Applies appropriate rate (£2,000 or £4,000 yearly)
     - Ensures quarterly caps (£500 or £1,000)
     - Sums total household entitlement